# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""97953358878468…","""../../../../da…","""15306907386400…",1.2927e19,"""73063131062807…","""12041664342669…",false
"""72129039474782…","""../../../../da…","""35025951945250…",1.4856e19,"""73063131062807…","""75270469056872…",true
"""18388240598088…","""../../../../da…","""33689831625072…",5.6458e17,"""75786297548846…","""12041664342669…",false
"""68135673984751…","""../../../../da…","""35025951945250…",4.0723e18,"""13053240249232…","""12041664342669…",false
"""47697503424586…","""../../../../da…","""15306907386400…",1.4677e19,"""13053240249232…","""10690259351510…",true
"""94478656701119…","""../../../../da…","""33689831625072…",1.6330e19,"""43456233287197…",null,false
"""12021642079192…","""../../../../da…","""33689831625072…",8.7675e18,"""13053240249232…","""10690259351510…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""17746492844631…","""../../../../da…","""17571988744947…",1.2927e19,"""FirstMortgage3…","""17087839121206…","""Rejected"""
"""11133059475251…","""../../../../da…","""29786406665359…",1.4856e19,"""FirstMortgage3…","""11249277848636…","""Accepted"""
"""15145823863637…","""../../../../da…","""14038309077624…",5.6458e17,"""MoneyMarketSav…","""17087839121206…","""Rejected"""
"""10361079862701…","""../../../../da…","""29786406665359…",4.0723e18,"""BasicChecking""","""17087839121206…","""Rejected"""
"""27202283909085…","""../../../../da…","""17571988744947…",1.4677e19,"""BasicChecking""","""53510716974339…","""Accepted"""
"""46351702187466…","""../../../../da…","""14038309077624…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""16538424244133…","""../../../../da…","""14038309077624…",8.7675e18,"""BasicChecking""","""53510716974339…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""12766913796236…","""../../../../da…","""74584698300116…",1.2927e19,"""FirstMortgage3…","""39016048750216…","""Rejected"""
"""68292852418638…","""../../../../da…","""43915497384262…",1.4856e19,"""FirstMortgage3…","""17539660301932…","""Accepted"""
"""15142153048104…","""../../../../da…","""51524465992013…",5.6458e17,"""MoneyMarketSav…","""39016048750216…","""Rejected"""
"""14224502227668…","""../../../../da…","""43915497384262…",4.0723e18,"""BasicChecking""","""39016048750216…","""Rejected"""
"""10366304805496…","""../../../../da…","""74584698300116…",1.4677e19,"""BasicChecking""","""15629747151437…","""Accepted"""
"""17009230294980…","""../../../../da…","""51524465992013…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""12663269376769…","""../../../../da…","""51524465992013…",8.7675e18,"""BasicChecking""","""15629747151437…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""10668049608346…","""10143286345217…",1.2927e19,"""FirstMortgage3…","""13736873101588…","""Rejected"""
"""25996158122413…","""13791402545406…",1.4856e19,"""FirstMortgage3…","""53316837518277…","""Accepted"""
"""17182244835788…","""10716954374874…",5.6458e17,"""MoneyMarketSav…","""13736873101588…","""Rejected"""
"""17226335744641…","""13791402545406…",4.0723e18,"""BasicChecking""","""13736873101588…","""Rejected"""
"""77932801077034…","""10143286345217…",1.4677e19,"""BasicChecking""","""90431188539059…","""Accepted"""
"""36401091832888…","""10716954374874…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""17656235665901…","""10716954374874…",8.7675e18,"""BasicChecking""","""90431188539059…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'